In [2]:
import pandas, re, cleantext, nltk

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
# andreas
def stem(input):
  stemmer = nltk.PorterStemmer()
  for token in input:
    stemmer.stem(token)

stem("news_sample.csv")

TypeError: StemmerI.stem() missing 1 required positional argument: 'token'

In [ ]:
# aidin
import pandas, re, cleantext, nltk



In [ ]:
# chrisser